# Correlation Network

Correlation network (graph) desribes the strength of correlation among qubit pairs in a system and shows how qubits in a system is correlated. 

In the network:

Nodes: represent the *qubits*.

Edge: represent *how much* the nodes (qubits) are *correlated* by its weight.

The strength can be any value that quantify how a qubit pair is correlated. In previous studies, quantum Mutual Information (MI) is used. Though it remains unknown, we conjecture that Entanglement of Formation (EoF) and various entanglement witnesses are also useful quantities in certain scenario. 

In quantum computing, people can calculate the correlation network of a objective wavefunction, such as the network of the ground state of a Hamiltonian in VQE, in a approximated way, before carrying out quantum computing on the system.

By this pre-calculation, people can know that

- Which qubits are more active and which qubits are nearly stationary and can be removed.
- The groups of qubits in which the qubits are inter-entangled, so that the computation can be divided into parts.
- Which part of the system is more active and needs higher precision of operation. 
    - So that physics qubits with higher quality can be assigned to it.

We believe that the potential applications shown above make pre-calculation of correlation network very important for near-term quantum computing because

- The number of gates and qubits allowed on near-term quantum devices is very limited.
- The quality of qubits in near-term is not uniform and full-connectivity qubits set will be small.

To address these problems, we provide modules for

- Easily carrying out correlation network approximation from qubit Hamiltonians.
- Finding the optimal mapping to physical qubits based on the correlation network.
- Finding the subsets of qubits which are most correlated.
- Finding the community (correlated group) structure of a correlation network.

## Classical pre-calculation

### Dependency

The classcial pre-calculation module in Mizore is realized by [dmrgpy](https://github.com/joselado/dmrgpy) (iTensor inside). The users must install dmrgpy before using this module. We remind the users that dmrgpy is based on iTensor and `lapack` is needed before installing. The ubuntu users can use `sudo apt-get install libblas-dev liblapack-dev` to get `lapack` ready.

### Basic Usage

The classical pre-calculation can be easily carried out as following.

In [ ]:
from mizore.Precalculation.iTensorCore import run_classcal_precalculation
from mizore.HamiltonianGenerator import make_example_H2O

# Generate the problem Hamiltonian
energy_obj = make_example_H2O()

# Run the classical pre-calculation
classical_res=run_classcal_precalculation(energy_obj.n_qubit,energy_obj.hamiltonian,calc_2DM=True)
print("Energy",classical_res["energy"])
print("Entropy",classical_res["entropy"])

Because `calc_2DM` is made `True`, reduced two-qubit density matrices of the ground state is also calculated. Here we show how to use them calculated the correlation quantities.

In [ ]:
from mizore.Utilities.WaveLocalProperties import get_mutual_information_by_2DMs,get_EoF_by_2DMs
 
classical_res["MI"]=get_mutual_information_by_2DMs(classical_res["2DM"])
classical_res["EoF"]=get_EoF_by_2DMs(classical_res["2DM"])

print("Mutual Information",classical_res["MI"].tolist())
print("Entanglement of Formation",classical_res["EoF"].tolist())

In [ ]:
from mizore.HamiltonianGenerator import make_example_LiH
from mizore.Precalculation.iTensorCore import run_classcal_precalculation
from mizore.Utilities.WaveLocalProperties import get_mutual_information_by_2DMs,get_EoF_by_2DMs

# Generate the problem Hamiltonian
energy_obj = make_example_LiH()

# Run the classical pre-calculation
classical_res=run_classcal_precalculation(energy_obj.n_qubit,
                                          energy_obj.hamiltonian,calc_2DM=True)
print("Energy",classical_res["energy"])
print("Entropy",classical_res["entropy"])

# Calculate and print pairwise qubit correlations
MI=get_mutual_information_by_2DMs(classical_res["2DM"]).tolist()
EoF=get_EoF_by_2DMs(classical_res["2DM"]).tolist()
print("Mutual Information",MI)
print("Entanglement of Formation",EoF)

We can convert the weight matrix to a network and visulize it as following.

In [ ]:
from mizore.CorrelationNetwork import get_nx_graph_by_adjacent_mat, draw_graph

G = get_nx_graph_by_adjacent_mat(classical_res["MI"])
draw_graph(G)

## Most correlated subsets

Which groups of qubits are most correlated? Mizore provides functions to estimate the most correlated subsets of qubits from the correlation network by Genetic Algorithm (GA). The subsets are generated by optimizing the fitness function
$$fitness(nodes) = \frac{\sum weight(edge)}{\sum num(nodes)*(num(nodes)-1)}$$

In [1]:
corr_adjacent_mat=[[0.0, 0.05686422974791204, 0.01688342559421084, 0.00923050347926574, 0.0438784337237768, 0.043615348773295606], [0.05686422974791204, 0.0, 0.00923050347926574, 0.016984823342198085, 0.043615348773295606, 0.043794613240691074], [0.01688342559421084, 0.00923050347926574, 0.0, 0.0021556255819239617, 0.0012450419319145478, 0.006317482790596407], [0.00923050347926574, 0.016984823342198085, 0.0021556255819239617, 0.0, 0.006317482790596407, 0.0014123307378750144], [0.0438784337237768, 0.043615348773295606, 0.0012450419319145478, 0.006317482790596407, 0.0, 0.03286268696855375], [0.043615348773295606, 0.043794613240691074, 0.006317482790596407, 0.0014123307378750144, 0.03286268696855375, 0.0]]

In [ ]:
from mizore.CorrelationNetwork.MostCorrelation._ga_selector import GACorrelationQsubsetSelector
from mizore.CorrelationNetwork._quantum_chips import *
from mizore.CorrelationNetwork import get_nx_graph_by_adjacent_mat

# Initial graph
G_corr = get_nx_graph_by_adjacent_mat(corr_adjacent_mat)

selector = GACorrelationQsubsetSelector(G_corr)

# Run MostCorrelation selector with time budget 10 seconds
selector.run(time_budget=10)

# Get Result
corr_qsubset = selector.get_result()

print(corr_qsubset)

After obtaining the most correlated subsets, we can use them to construct a block pool with blocks all act on the subsets, or reduce a block pool by screening the blocks who do not act on a very correlated subset. The strategy was proposed in [Mutual Information-assisted adaptive VQE](https://arxiv.org/abs/2008.07553).

In [ ]:
from mizore.PoolGenerator._qsubset_pools import get_qsubset_pool_reduced_block_pool
from mizore.PoolGenerator import BlockPool,all_rotation_pool
pool=BlockPool(all_rotation_pool(6))
reduced_pool=get_qsubset_pool_reduced_block_pool(pool,corr_qsubset)
print(reduced_pool)

## Community detection

Community detection is important for near-term quantum computing. The importance has been shown by previous works like [arXiv:1904.00102](http://arxiv.org/abs/1904.00102) and [arXiv:2007.00958v1](http://arxiv.org/abs/2007.00958v1).

Here we show how to detect community of qubits by its correlation network by Mizore.

In [2]:
from mizore.CorrelationNetwork._community_detection import detect_nx_graph_community, draw_community_graph
from mizore.CorrelationNetwork import get_nx_graph_by_adjacent_mat
import matplotlib
matplotlib.use("PDF")
# Initial graph
G_corr = get_nx_graph_by_adjacent_mat([[0.0, 0.0016703871993082597, 0.001225305394286537, 0.0001590563955301133, 0.02439288817202466, 0.011008468796831226, 0.024329061268075016, 0.0015228928210996515, 0.0007177029604427537, 0.0015228928210996515], [0.0016703871993082597, 0.0, 0.0016790225502884663, 0.0006769553917860332, 0.0013796056417296337, 0.003735582615582232, 0.00027252491442756807, 0.006425398206410247, 0.0017293892715447776, 0.006425398206410247], [0.001225305394286537, 0.0016790225502884663, 0.0, 1.1626592484253323e-05, 0.002429437418140043, 0.0014491824768585182, 0.0013672247368685872, 0.0017408020706285686, 0.037732699260398794, 0.0017408020706285686], [0.0001590563955301133, 0.0006769553917860332, 1.1626592484253323e-05, 0.0, 0.00013448786061091844, 0.0003182380771297212, 2.7744269142759848e-05, 2.9874857936031292e-05, 1.0696220814170043e-05, 2.9874857936031292e-05], [0.02439288817202466, 0.0013796056417296337, 0.002429437418140043, 0.00013448786061091844, 0.0, 0.007960096390564254, 0.00015471065364042885, 0.0012572765294868657, 0.0006691853998998104, 0.0012572765294868657], [0.011008468796831226, 0.003735582615582232, 0.0014491824768585182, 0.0003182380771297212, 0.007960096390564254, 0.0, 0.006299924267564774, 0.003372615812149333, 0.0008321710701455626, 0.003372615812149333], [0.024329061268075016, 0.00027252491442756807, 0.0013672247368685872, 2.7744269142759848e-05, 0.00015471065364042885, 0.006299924267564774, 0.0, 0.00024506350979397115, 0.0002507214624983961, 0.00024506350979397115], [0.0015228928210996515, 0.006425398206410247, 0.0017408020706285686, 2.9874857936031292e-05, 0.0012572765294868657, 0.003372615812149333, 0.00024506350979397115, 0.0, 0.0017968706304769579, 0.006831027487169679], [0.0007177029604427537, 0.0017293892715447776, 0.037732699260398794, 1.0696220814170043e-05, 0.0006691853998998104, 0.0008321710701455626, 0.0002507214624983961, 0.0017968706304769579, 0.0, 0.0017968706304769579], [0.0015228928210996515, 0.006425398206410247, 0.0017408020706285686, 2.9874857936031292e-05, 0.0012572765294868657, 0.003372615812149333, 0.00024506350979397115, 0.006831027487169679, 0.0017968706304769579, 0.0]])

# Detect community
community_map = detect_nx_graph_community(G_corr)
print(community_map)

# Draw community graph
draw_community_graph(G_corr, community_map, "output0")

{9: 1, 8: 1, 7: 1, 5: 2, 6: 2, 2: 2, 4: 2, 1: 2, 0: 2, 3: 3}


## Optimal Mapping


### Dependency

The optimal mapping module in Mizore is realized by [minorminer](https://github.com/dwavesystems/minorminer).  The ubuntu users can use `pip3 install minorminer` to get `minorminer` ready.

### Basic Usage

Optimal Mapping (graph) desribes the graph embedding of source graph (correlation network) in target graph (Connectivity graph of the physical qubits).

We feed the generic constructor with both source weighted graph and target weighted graph. The optimal results will be evolved and optimized though a heuristic and generic hybrid algorithm. (Fitness will be evaluated by: $fitness(source\_graph, embedding\_graph) = \sum weight(edge_{source\_graph}) * weight(edge_{embeding\_graph})$ )

In [ ]:
from mizore.CorrelationNetwork import GAGraphEmbeddingConstructor
from mizore.CorrelationNetwork._quantum_chips import *

# Initial quantum chips
rigetti_16Q_Aspen = Rigetti_16Q_Aspen()
ibm_20Q_Johannesburg = IBM_20Q_Johannesburg()

# Search optimal mapping
embeding_selector = GAGraphEmbeddingConstructor(rigetti_16Q_Aspen, ibm_20Q_Johannesburg)
embeding_selector.run(time_budget=5)

# Show optimal mapping
results = embeding_selector.get_result()
best_result = results[0]
print(f'fitness :{best_result.fitness}, mapping:{[gene for gene in best_result.genes]}')

## Hardward Restricted Pool

Here we show how to generate a block pool in which the operators all act on a path of a graph (Connectivity graph of the physical qubits). By doing so we can avoid the compiling overhead when mapping blocks not on the path to devices. 

In [ ]:
from mizore.CorrelationNetwork._quantum_chips import *
from mizore.CorrelationNetwork._utilities import find_paths

# Initial quantum chip
chip = Rigetti_8Q_Agave()
# Find paths
paths = find_paths(chip)
print(paths)

from mizore.PoolGenerator._qsubset_pools import iter_entangler_by_qsubsets
from mizore.Blocks._efficient_coupled_cluster import EfficientCoupledCluster
from mizore.PoolGenerator import BlockPool

pool = BlockPool(iter_entangler_by_qsubsets(
    paths, EfficientCoupledCluster))
print(pool)